# Dev [Part 4 - Set Home and Position One]

In [1]:
%pylab inline
import sys
from ipywidgets import interact

sys.path.append('src')
from motorControl import *
from trajectoryPlanning import *

rc = connect(portName = "/dev/tty.usbserial-A9ETDN3N")

Populating the interactive namespace from numpy and matplotlib


### Initialized Motor Classes

- Really important to initialze these first to restore positions from last session!

In [2]:
#Initialize motor objects for each motor:
vL = Motor(address = 0x81, motorNumber = 2, rc = rc, signFlipped = False, \
           motorCounter = 0, kPID = [1e-2, 1.0])
vR = Motor(address = 0x81, motorNumber = 1, rc = rc, signFlipped = False, \
           motorCounter = 1, kPID = [1e-2, 1.0])
LR = Motor(address = 0x80, motorNumber = 1, rc = rc, signFlipped = False, 
           motorCounter = 2, kPID = [1e-2, 1.0])
FB = Motor(address = 0x80, motorNumber = 2, rc = rc, signFlipped = True, \
           motorCounter = 3, kPID = [1e-2, 1.0])
yaw = Motor(address = 0x82, motorNumber = 2, rc = rc, signFlipped = False, \
            motorCounter = 4, kPID = [1e-2, 1.0])
pitch = Motor(address = 0x82, motorNumber = 1, rc = rc, signFlipped = False, \
              motorCounter = 5, kPID = [1e-2, 1.0])

#Keep in a nice motor list:
motors = [vL, vR, LR, FB, yaw, pitch]

In [3]:
savePositions(motors)
getPositions(motors)

[-416, -24779, 53468, 1458, 8125, 4406]

In [67]:
interact(manualControl, leftUD = (-50, 50), rightUD = (-50, 50), leftRight = (-50, 50), \
         fB = (-50, 50), tilt = (-50, 50), pan = (-50, 50))

In [68]:
stopAll(rc)
savePositions(motors)
getPositions(motors)

[-64236, 64853, -132500, 60496, 22981, 11495]

In [ ]:
# for motor in motors:
#     motor.resetEncoders()

In [25]:
# f = open('savedPositions/home.p', 'wb')
# pickle.dump(getPositions(motors), f, protocol=pickle.HIGHEST_PROTOCOL)
# f.close()

In [69]:
# f = open('savedPositions/position2.p', 'wb')
# pickle.dump(getPositions(motors), f, protocol=pickle.HIGHEST_PROTOCOL)
# f.close()

In [41]:
# f = open('savedPositions/position1.p', 'wb')
# pickle.dump(getPositions(motors), f, protocol=pickle.HIGHEST_PROTOCOL)
# f.close()

In [42]:
f = open('savedPositions/home.p', 'r')
home = pickle.load(f)
f.close()

f = open('savedPositions/position1.p', 'r')
position1 = pickle.load(f)
f.close()

In [43]:
print position1, home

[0, 0, -69159, 58530, 0, 45296] [0, 0, 0, 0, 0, 0]


In [52]:
targetPositions = position1
totalTime = 40.0
rampTime = 5.0

startingPositions = getPositions(motors)
lookAheadTime = 1.0
tolerance = 100.0 #Anything less than this many ticks we're calling "Not a move"

motorsToMove = []
targetPositionsToMove = []
for i, motor in enumerate(motors):
    if abs(motor.getPosition()-targetPositions[i]) > tolerance:
        motorsToMove.append(motor)
        targetPositionsToMove.append(targetPositions[i])
        
trajectories = []
for i, motor in enumerate(motorsToMove):
    trajectories.append(SimpleQuadraticTrajectory(tu = rampTime, tt = totalTime, \
                                            p1 = motor.getPosition(), p2 = targetPositionsToMove[i]))

In [53]:
for motor in motorsToMove:
    motor.initialize(targetVelocityMin = -2500.0, targetVelocityMax = 2500.0)
    motor.clearTracking()
    
startTime = time.time()
timeElapsed = 0.0
    
while timeElapsed < totalTime:
    timeElapsed = time.time()-startTime
    
    for i, motor in enumerate(motorsToMove):
        lookAheadValue = trajectories[i].compute(timeElapsed + lookAheadTime)
        motor.controlledMove(targetPosition = lookAheadValue, timeToReach = lookAheadTime)

stopAll(rc)
savePositions(motors)
print getPositions(motors), targetPositions
print getPositions(motors) - np.array(targetPositions)

[0, 0, -69010, 58399, 0, 45193] [0, 0, -69159, 58530, 0, 45296]


### Ok, so now the game is wrapping things up into a nice, consistent, repeatable workflow!

- Make an "absolute" zero that makes sense, reset encoders and save!
    - maybe even make a "goHome" method or something. 
    - This will be our first stored "absolute way point"
    - I think home should be a position that is as repeatable as possible, and out of th eway, maybe far back in up righ corner. I'll take a picture, and put it in the repository. Oooh, and I need to check the levelness of the horizontal beam for this point!
    - It's that's home, then one of my positions should "centered", that is directly overhead, looking straight down. These will be my first two absolute positions. 
- From there, I wanto to be able to create, save, and name new absolute positions using the manual control sliders. 
- Finally, I want a nice, wrapped up way of of moving between positions!
- That should be all I need to HTS 1. 
- Next time I work on motion rig - "keep the camera looking at the same thing" code. That will be fun!